In [1]:
#importing required libraries
import os
import shutil
import time

import folium
import imageio
import webbrowser
import zipfile
import json
import fileinput

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from os import path
from branca.colormap import linear
from selenium import webdriver
from PIL import Image
from pathlib import Path

from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap

In [2]:
#reading 2016 data into respective dataframes
df1 = pd.read_csv('../Data/2016/Divvy_Trips_2016_Q1.csv')
df2 = pd.read_csv('../Data/2016/Divvy_Trips_2016_04.csv')
df3 = pd.read_csv('../Data/2016/Divvy_Trips_2016_05.csv')
df4 = pd.read_csv('../Data/2016/Divvy_Trips_2016_06.csv')
df5 = pd.read_csv('../Data/2016/Divvy_Trips_2016_Q3.csv')
df6 = pd.read_csv('../Data/2016/Divvy_Trips_2016_Q4.csv')

In [3]:
#Concatenating the dataframes into a single dataframe
frames = [df1,df2,df3,df4,df5,df6]
dfbike = pd.concat(frames)
dfbike.reset_index(drop=True,inplace=True)
dfbike.head(5)

,trip_id,starttime,stoptime,bikeid,tripduration,from_station_id,from_station_name,to_station_id,to_station_name,usertype,gender,birthyear
0,9080551,3/31/2016 23:53,4/1/2016 0:07,155,841,344,Ravenswood Ave & Lawrence Ave,458,Broadway & Thorndale Ave,Subscriber,Male,1986.0
1,9080550,3/31/2016 23:46,3/31/2016 23:57,4831,649,128,Damen Ave & Chicago Ave,213,Leavitt St & North Ave,Subscriber,Male,1980.0
2,9080549,3/31/2016 23:42,3/31/2016 23:46,4232,210,350,Ashland Ave & Chicago Ave,210,Ashland Ave & Division St,Subscriber,Male,1979.0
3,9080548,3/31/2016 23:37,3/31/2016 23:55,3464,1045,303,Broadway & Cornelia Ave,458,Broadway & Thorndale Ave,Subscriber,Male,1980.0
4,9080547,3/31/2016 23:33,3/31/2016 23:37,1750,202,334,Lake Shore Dr & Belmont Ave,329,Lake Shore Dr & Diversey Pkwy,Subscriber,Male,1969.0


In [4]:
dfbike.columns

Index(['trip_id', 'starttime', 'stoptime', 'bikeid', 'tripduration',
       'from_station_id', 'from_station_name', 'to_station_id',
       'to_station_name', 'usertype', 'gender', 'birthyear'],
      dtype='object')

### Saving all the station names in a list "allSt16"

#### Comparing with 2015 data and only adding the stations which are new into a list "added16"
#### Also, checking for the stations that have been in 2015 but not in this year's data and adding it to "removed15"

In [5]:
allSt16 = list(dfbike['from_station_name'].unique())
len(allSt16)

587

In [6]:
allSt15 = ['Wilton Ave & Belmont Ave',
 'Michigan Ave & Washington St',
 'Damen Ave & Wellington Ave',
 'Clark St & Randolph St',
 'Peoria St & Jackson Blvd',
 'Broadway & Belmont Ave',
 'Sheffield Ave & Webster Ave',
 'Sheffield Ave & Wellington Ave',
 'Wells St & Erie St',
 'Ashland Ave & Division St',
 'Sheffield Ave & Kingsbury St',
 'Racine Ave & Belmont Ave',
 'Halsted St & Roscoe St',
 'Ogden Ave & Race Ave',
 'Green St & Randolph St',
 'Ravenswood Ave & Lawrence Ave',
 'Millennium Park',
 'Western Ave & Division St',
 'Southport Ave & Clybourn Ave',
 'Paulina St & Montrose Ave',
 'Clark St & Chicago Ave',
 'Halsted St & Diversey Pkwy',
 'Claremont Ave & Hirsch St',
 'Damen Ave & Pierce Ave',
 'Western Ave & Winnebago Ave',
 'Damen Ave & Charleston St',
 'Lincoln Ave & Leavitt St',
 'Hampden Ct & Diversey Pkwy',
 'Clark St & Elm St',
 'Canal St & Madison St',
 'Wells St & Hubbard St',
 'Lincoln Ave & Roscoe St',
 'Canal St & Adams St',
 'Damen Ave & Division St',
 'Ogden Ave & Chicago Ave',
 'Clark St & Schiller St',
 'Michigan Ave & Lake St',
 'Loomis St & Taylor St',
 'Canal St & Monroe St',
 'Sheridan Rd & Irving Park Rd',
 'Dayton St & North Ave',
 'State St & Harrison St',
 'Halsted St & Polk St',
 'Ashland Ave & Chicago Ave',
 'Kedzie Ave & Milwaukee Ave',
 'Calumet Ave & 33rd St',
 'Emerald Ave & 31st St',
 'Southport Ave & Waveland Ave',
 'Pine Grove Ave & Irving Park Rd',
 'State St & Randolph St',
 'Racine Ave & Congress Pkwy',
 'Clark St & Wellington Ave',
 'Southport Ave & Belmont Ave',
 'Lincoln Ave & Waveland Ave',
 'Sheffield Ave & Addison St',
 'Broadway & Sheridan Rd',
 'Lake Shore Dr & Wellington Ave',
 'Lake Shore Dr & Diversey Pkwy',
 'Wabash Ave & Grand Ave',
 'Blue Island Ave & 18th St',
 'Canal St & Jackson Blvd',
 'Daley Center Plaza',
 'Ravenswood Ave & Irving Park Rd',
 'Michigan Ave & Pearson St',
 'May St & Fulton St',
 'Woodlawn Ave & 55th St',
 'Franklin St & Chicago Ave',
 'Fairbanks Ct & Grand Ave',
 'Halsted St & 21st St',
 'Dearborn Pkwy & Delaware Pl',
 'Franklin St & Lake St',
 'Carpenter St & Huron St',
 'Clark St & Wrightwood Ave',
 'Wells St & Evergreen Ave',
 'Broadway & Waveland Ave',
 'Damen Ave & Cortland St',
 'Wells St & Concord Ln',
 'Sheffield Ave & Fullerton Ave',
 'Halsted St & Madison St',
 'State St & Van Buren St',
 'Clark St & Leland Ave',
 'Rhodes Ave & 32nd St',
 'Wabash Ave & Roosevelt Rd',
 'Orleans St & Merchandise Mart Plaza',
 'Sheridan Rd & Montrose Ave',
 'Damen Ave & Chicago Ave',
 'May St & Randolph St',
 'Wilton Ave & Diversey Pkwy',
 'MLK Jr Dr & 47th St',
 'Dearborn St & Monroe St',
 'Sedgwick St & North Ave',
 'May St & Taylor St',
 'Sheffield Ave & Wrightwood Ave',
 'Wells St & Walton St',
 'California Ave & 21st St',
 'LaSalle St & Washington St',
 'Michigan Ave & Jackson Blvd',
 'Clinton St & Madison St',
 'Ravenswood Ave & Montrose Ave',
 'Dearborn St & Erie St',
 'Lake Shore Dr & Monroe St',
 'Southport Ave & Roscoe St',
 'Southport Ave & Irving Park Rd',
 'Pine Grove Ave & Waveland Ave',
 'Franklin St & Jackson Blvd',
 'Kingsbury St & Erie St',
 'California Ave & Division St',
 'Wood St & Milwaukee Ave',
 'Leavitt St & Armitage Ave',
 'Cannon Dr & Fullerton Ave',
 'Streeter Dr & Illinois St',
 'Southport Ave & Wellington Ave',
 'Aberdeen St & Jackson Blvd',
 '900 W Harrison St',
 'Ashland Ave & Wellington Ave',
 'Halsted St & Wrightwood Ave',
 'Halsted St & James M Rochford St',
 'Western Ave & Leland Ave',
 'Sheffield Ave & Willow St',
 'Damen Ave & Grand Ave',
 'Morgan St & Lake St',
 'Damen Ave & Leland Ave',
 'Clark St & Winnemac Ave',
 'Broadway & Barry Ave',
 'Ashland Ave & Grand Ave',
 'Cityfront Plaza Dr & Pioneer Ct',
 'Milwaukee Ave & Wabansia Ave',
 'Lake Shore Dr & North Blvd',
 'Morgan St & 18th St',
 'Mies van der Rohe Way & Chicago Ave',
 'St. Clair St & Erie St',
 'Clark St & Grace St',
 'Halsted St & Blackhawk St',
 'Clark St & Montrose Ave',
 'Wolcott Ave & Polk St',
 'LaSalle St & Illinois St',
 'Clarendon Ave & Gordon Ter',
 'Bissell St & Armitage Ave',
 'Lake Park Ave & 56th St',
 'State St & Pearson St',
 'Wabash Ave & 8th St',
 'Larrabee St & Kingsbury St',
 'McClurg Ct & Illinois St',
 'Morgan St & Polk St',
 'Franklin St & Quincy St',
 'Sedgwick St & Webster Ave',
 'Lincoln Ave & Belmont Ave',
 'Ashland Ave & 21st St',
 'Michigan Ave & Congress Pkwy',
 'State St & 33rd St',
 'Larrabee St & Webster Ave',
 'Lakeview Ave & Fullerton Pkwy',
 'Aberdeen St & Madison St',
 'Loomis St & Lexington St',
 'Ashland Ave & Wrightwood Ave',
 'Ashland Ave & 13th St',
 'Halsted St & Archer Ave',
 'Clark St & Lincoln Ave',
 'Clark St & Congress Pkwy',
 'Halsted St & Dickens Ave',
 'Franklin St & Arcade Pl',
 'Ravenswood Ave & Berteau Ave',
 'Sangamon St & Washington Blvd',
 'Dearborn St & Adams St',
 'Desplaines St & Kinzie St',
 'Racine Ave & 15th St',
 'Sedgwick St & Huron St',
 'Greenview Ave & Fullerton Ave',
 'Eckhart Park',
 'Emerald Ave & 28th St',
 'Leavitt St & North Ave',
 'Lincoln Ave & Fullerton Ave',
 'Racine Ave & Fullerton Ave',
 'Southport Ave & Wrightwood Ave',
 'Michigan Ave & 14th St',
 'Stave St & Armitage Ave',
 'California Ave & Milwaukee Ave',
 'Damen Ave & Augusta Blvd',
 'State St & Kinzie St',
 'Broadway & Cornelia Ave',
 'Wells St & Polk St',
 'Paulina St & 18th St',
 'Wood St & North Ave',
 'Broadway & Wilson Ave',
 'Logan Blvd & Elston Ave',
 'Kedzie Ave & Palmer Ct',
 'Clark St & North Ave',
 'Stetson Ave & South Water St',
 'Sheridan Rd & Buena Ave',
 'Ashland Ave & Belle Plaine Ave',
 'Halsted St & 37th St',
 'Clarendon Ave & Junior Ter',
 'Clifton Ave & Lawrence Ave',
 'Clinton St & Washington Blvd',
 'Clark St & Armitage Ave',
 'Halsted St & 35th St',
 'Michigan Ave & 16th St',
 'Michigan Ave & Oak St',
 'LaSalle St & Jackson Blvd',
 'Columbus Dr & Randolph St',
 'Ashland Ave & Blackhawk St',
 'Racine Ave & Wrightwood Ave',
 'Larrabee St & North Ave',
 'Clinton St & Tilden St',
 'Union Ave & Grand Ave',
 'Broadway & Berwyn Ave',
 'Halsted St & Willow St',
 'Lincoln Ave & Addison St',
 'Ellis Ave & 58th St',
 'Indiana Ave & Roosevelt Rd',
 'Theater on the Lake',
 'Lake Shore Dr & Belmont Ave',
 'California Ave & Francis Pl',
 'Sedgwick St & Schiller St',
 'Clinton St & Lake St',
 'Cottage Grove Ave & 51st St',
 'Greenview Ave & Diversey Pkwy',
 'Clybourn Ave & Division St',
 'Leavitt St & Hirsch St',
 'Wabash Ave & Wacker Pl',
 'Damen Ave & Sunnyside Ave',
 'Calumet Ave & 18th St',
 'Michigan Ave & Madison St',
 'Seeley Ave & Roscoe St',
 'Ashland Ave & Lake St',
 'Shore Drive & 55th St',
 'Lincoln Ave & Diversey Pkwy',
 'Clarendon Ave & Leland Ave',
 'Montrose Harbor',
 'Southport Ave & Clark St',
 'Ashland Ave & Augusta Blvd',
 'Ogden Ave & Congress Pkwy',
 'Noble St & Milwaukee Ave',
 'McCormick Place',
 'Cottage Grove Ave & 47th St',
 'Campbell Ave & North Ave',
 'Lincoln Ave & Belle Plaine Ave',
 'Marshfield Ave & Cortland St',
 'State St & 19th St',
 'Prairie Ave & Garfield Blvd',
 'Indiana Ave & 26th St',
 'Canal St & Harrison St',
 'Morgan Ave & 14th Pl',
 'Fort Dearborn Dr & 31st St',
 'Wentworth Ave & Archer Ave',
 'Museum Campus',
 'Hermitage Ave & Polk St',
 'Milwaukee Ave & Rockwell St',
 'State St & 35th St',
 'Leavitt St & Lawrence Ave',
 'Clifton Ave & Armitage Ave',
 'Wood St & Grand Ave',
 'Clinton St & Roosevelt Rd',
 'California Ave & North Ave',
 'Lake Shore Dr & Ohio St',
 'Damen Ave & Madison St',
 'Wood St & Taylor St',
 'Adler Planetarium',
 'Wood St & Division St',
 'Stockton Dr & Wrightwood Ave',
 'Halsted St & Maxwell St',
 'Ada St & Washington Blvd',
 'Racine Ave & 13th St',
 'MLK Jr Dr & 29th St',
 'Clinton St & 18th St',
 'Larrabee St & Menomonee St',
 'Loomis St & Jackson Blvd',
 'Racine Ave & 18th St',
 'Kimbark Ave & 53rd St',
 'Jefferson St & Monroe St',
 'Lake Park Ave & 47th St',
 'Orleans St & Elm St',
 'Normal Ave & Archer Ave',
 'Larrabee St & Armitage Ave',
 'Ashland Ave & Grace St',
 'Blackstone Ave & Hyde Park Blvd',
 'Morgan St & 31st St',
 'Wentworth Ave & 24th St',
 'Damen Ave & Cullerton St',
 'Damen Ave & Melrose Ave',
 'Indiana Ave & 31st St',
 'Cottage Grove Ave & Oakwood Blvd',
 'State St & 16th St',
 'Sheridan Rd & Lawrence Ave',
 'Halsted St & 18th St',
 'Greenwood Ave & 47th St',
 'Cottage Grove Ave & 43rd St',
 'Wallace St & 35th St',
 'Calumet Ave & 35th St',
 'Western Ave & 24th St',
 'Damen Ave & Clybourn Ave',
 'May St & Cullerton St',
 'Wabash Ave & Cermak Rd',
 'Broadway & Argyle St',
 'Wells St & 19th St',
 'Western Ave & 21st St',
 'State St & 29th St',
 'MLK Jr Dr & Oakwood Blvd',
 'Damen Ave & Coulter St',
 'Indiana Ave & 40th St',
 'Troy St & Elston Ave',
 'Princeton Ave & 31st St',
 'Wabash Ave & Adams St',
 'Rush St & Superior St',
 'Clinton St & Polk St',
 'Desplaines St & Randolph St',
 'Rush St & Hubbard St',
 'Damen Ave & Foster Ave',
 'Ashland Ave & Harrison St',
 'LaSalle St & Adams St',
 'Kimball Ave & Belmont Ave',
 'Western Ave & Granville Ave',
 'Drake Ave & Addison St',
 'Francisco Ave & Foster Ave',
 'Monticello Ave & Irving Park Rd',
 'Clark St & Schreiber Ave',
 'Federal St & Polk St',
 'Kingsbury St & Kinzie St',
 'Manor Ave & Leland Ave',
 'Campbell Ave & Fullerton Ave',
 'Glenwood Ave & Morse Ave',
 'Humboldt Blvd & Armitage Ave',
 'Clark St & Lake St',
 'Albany Ave & Bloomingdale Ave',
 'Campbell Ave & Montrose Ave',
 'Western Ave & Roscoe St',
 'University Ave & 57th St',
 'State St & Pershing Rd',
 'Maplewood Ave & Peterson Ave',
 'Dusable Harbor',
 '2112 W Peterson Ave',
 'Michigan Ave & Balbo Ave',
 'Broadway & Thorndale Ave',
 'Wentworth Ave & 33rd St',
 'Clark St & Berwyn Ave',
 'Burnham Harbor',
 'Prairie Ave & 43rd St',
 'Mies van der Rohe Way & Chestnut St',
 'Field Blvd & South Water St',
 'Spaulding Ave & Armitage Ave',
 'Shedd Aquarium',
 'Ellis Ave & 55th St',
 'Clark St & Bryn Mawr Ave',
 'Winchester Ave & Elston Ave',
 'Ravenswood Ave & Balmoral Ave',
 'Larrabee St & Division St',
 'Halsted St & North Branch St',
 'Wells St & Elm St',
 'Sawyer Ave & Irving Park Rd',
 'Larrabee St & Oak St',
 'Calumet Ave & 21st St',
 'Western Ave & Walton St',
 'Washtenaw Ave & Lawrence Ave',
 'Ridge Blvd & Touhy Ave',
 'Kedzie Ave & Leland Ave',
 'Kedzie Ave & Montrose Ave',
 'Ellis Ave & 53rd St',
 'Kedzie Ave & Lake St',
 'Loomis St & Archer Ave',
 'McClurg Ct & Erie St',
 'Christiana Ave & Lawrence Ave',
 'Talman Ave & Addison St',
 'Wacker Dr & Washington St',
 'Sheridan Rd & Greenleaf Ave',
 'Avers Ave & Belmont Ave',
 'Cottage Grove Ave & 67th St',
 'Clark St & Elmdale Ave',
 'California Ave & Altgeld St',
 'Rush St & Cedar St',
 'Financial Pl & Congress Pkwy',
 'Harper Ave & 59th St',
 'Stony Island Ave & 67th St',
 'Kedzie Ave & 24th St',
 'Millard Ave & 26th St',
 'Lake Park Ave & 53rd St',
 'Lakefront Trail & Bryn Mawr Ave',
 'Troy St & North Ave',
 'California Ave & 23rd Pl',
 'Clark St & Touhy Ave',
 'Shields Ave & 28th Pl',
 'Clark St & Lunt Ave',
 'Broadway & Granville Ave',
 'Sheridan Rd & Loyola Ave',
 'Kosciuszko Park',
 'Western Ave & Monroe St',
 'Ritchie Ct & Banks St',
 'Jeffery Blvd & 67th St',
 'Kedzie Ave & 21st St',
 'Stony Island Ave & 71st St',
 'St. Louis Ave & Balmoral Ave',
 'Rockwell St & Eastwood Ave',
 'Kedzie Ave & Bryn Mawr Ave',
 'Artesian Ave & Hubbard St',
 'Lincoln Ave & Winona St',
 'Ellis Ave & 60th St',
 'Cottage Grove Ave & 63rd St',
 'Central Park Ave & Ogden Ave',
 'Keeler Ave & Irving Park Rd',
 'Desplaines St & Jackson Blvd',
 'Broadway & Ridge Ave',
 'Museum of Science and Industry',
 'Cornell Ave & Hyde Park B lvd',
 'Oakley Ave & Irving Park Rd',
 'Western Ave & Congress Pkwy',
 'Leavitt St & Addison St',
 'California Ave & Augusta Blvd',
 'Central Park Ave & Elbridge Ave',
 '63rd St Beach',
 'Warren Park East',
 'Drake Ave & Fullerton Ave',
 'South Shore Dr & 71st St',
 'California Ave & Lake St',
 'Halsted St & 47th Pl',
 'Dorchester Ave & 63rd St',
 'California Ave & 26th St',
 'Ogden Ave & Roosevelt Rd',
 'South Shore Dr & 74th St',
 'Richmond St & Diversey Ave',
 'Central Park Ave & North Ave',
 'Racine Ave & 35th St',
 'California Ave & Roosevelt Rd',
 'Clark St & Columbia Ave',
 'Washtenaw Ave & 15th St',
 'Keystone Ave & Montrose Ave',
 'Kedzie Ave & Chicago Ave',
 'Yates Blvd & 75th St',
 'Princeton Ave & China Pl',
 'Wentworth Ave & 63rd St',
 'Calumet Ave & 51st St',
 'Dorchester Ave & 49th St',
 'Warren Park West',
 'Jeffery Blvd & 76th St',
 'Drake Ave & Montrose Ave',
 'Kedzie Ave & Foster Ave',
 'California Ave & Fletcher St',
 'Kedzie Ave & Roosevelt Rd',
 'MLK Jr Dr & 56th St',
 'Lake Park Ave & 35th St',
 'Budlong Woods Library',
 'Stony Island Ave & 64th St',
 'Pulaksi Rd & Eddy St',
 'Union Ave & 42nd St',
 'Princeton Ave & 47th St',
 'Spaulding Ave & Division St',
 'Western Ave & Lunt Ave',
 'Wood St & 35th St',
 'South Shore Dr & 67th St',
 'Leavitt St & Archer Ave',
 'Princeton Ave & Garfield Blvd',
 'Jeffery Blvd & 71st St',
 'Albany Ave & 26th St',
 'Ashland Ave & Archer Ave',
 'Sacramento Blvd & Franklin Blvd',
 'Shields Ave & 43rd St',
 'Western Ave & 28th St',
 'Woodlawn Ave & Lake Park Ave',
 'Stony Island Ave & 75th St',
 'Halsted St & 69th St',
 'Cottage Grove Ave & 71st St',
 'Calumet Ave & 71st St',
 'Kedzie Ave & Harrison St',
 'Perry Ave & 69th St',
 'Halsted St & 63rd St',
 'Saginaw Ave & Exchange Ave',
 'Rainbow Beach',
 'MLK Jr Dr & 63rd St',
 'Halsted St & 51st St',
 'Halsted St & 56th St',
 'Lawndale Ave & 23rd St',
 'Eberhart Ave & 61st St',
 'Franklin St & Monroe St',
 'Marine Dr & Ainslie St',
 'Wells St & Concord Pl',
 'Green St (Halsted St) & Madison St',
 'Shields Ave & 31st St',
 'Paulina  Ave (Wood St) & North Ave',
 'Buckingham Fountain',
 'Canal St & Taylor St',
 'Cornell Ave & Hyde Park Blvd',
 'Aberdeen St & Monroe St (Madison St)',
 'Michigan Ave & 18th St',
 'Humboldt Dr & Luis Munoz Marin Dr',
 'Wabash Ave (State St) & 16th St',
 'California Ave & Byron St',
 'Elston Ave & Wabansia Ave',
 'California Ave & Montrose Ave',
 'Halsted St & Roosevelt Rd',
 'DuSable Museum',
 'Fairfield Ave & Roosevelt Rd',
 'Pulaski Rd & Eddy St',
 'Clark St & 9th St (AMLI)']

In [7]:
removed15 = list(set(allSt15) - set(allSt16))
len(removed15)

34

In [8]:
added16 = list(set(allSt16) - set(allSt15))
len(added16)

130

# Plotting all added stations of 2016 and removed stations of 2015

In [9]:
#reading the 'final_stations.csv' into a dataframe
#using all the stations co-ordinates and comparing with each of the year's stations added/ removed list and adding locations 
#     to each of the list to plot yearly data.

final = pd.read_csv('final_stations.csv')

In [10]:
final.head(1)

,Unnamed: 0,Station,Latitude,Longitude
0,0,Ogden Ave & Race Ave,41.891795,-87.658751


In [11]:
#Removing the unnecessary column in the dataframe
final = final.drop('Unnamed: 0', axis =1)

In [12]:
final.head(1)

,Station,Latitude,Longitude
0,Ogden Ave & Race Ave,41.891795,-87.658751


In [13]:
#Converting the dataframe into a dictionary
# key : Station name and value: latitude and longitude

locations = dict()

for station in final['Station']:
    locations[station] = {'lat': final[final['Station'] == station]['Latitude'].to_list()[0], 'long': final[final['Station'] == station]['Longitude'].to_list()[0]}

#printing out the first item in the dictionary to check the key-value pair
list(locations.items())[0]

('Ogden Ave & Race Ave', {'lat': 41.891795, 'long': -87.658751})

In [14]:
#Adding co-ordinates to all the stations that are added this year
n16 = {k: locations[k] for k in locations.keys() & set(added16)}

#printing out the first item in the dictionary to check the key-value pair
list(n16.items())[0]

('State St & 79th St', {'lat': 41.750794, 'long': -87.624424})

In [15]:
#Converting the dictionary into dataframe
st16 = pd.DataFrame.from_dict(n16, orient='index')

In [16]:
#Resetting the index
st16.reset_index(inplace=True)

#Renaming the index column to Station name
st16 = st16.rename(columns = {'index':'Station Name'})

In [17]:
#Adding a column with values of color for every station
st16['color'] = st16.apply(lambda row: 'red', axis=1)

#printing to check the structure of the dataframe
st16.head(1)

,Station Name,lat,long,color
0,State St & 79th St,41.750794,-87.624424,red


In [18]:
removed15 = ['Canal St & Monroe St',
 'Clinton St & Polk St',
 'Cornell Ave & Hyde Park B lvd',
 'Washtenaw Ave & 15th St',
 'Wells St & Erie St',
 'Halsted St & 35th St',
 'State St & 16th St',
 'MLK Jr Dr & 56th St',
 'Princeton Ave & China Pl',
 'Leavitt St & Hirsch St',
 'Sangamon St & Washington Blvd',
 'Loomis St & Taylor St',
 'Buckingham Fountain',
 'California Ave & Augusta Blvd',
 'Franklin St & Arcade Pl',
 'Wabash Ave (State St) & 16th St',
 'Orleans St & Elm St',
 'Wells St & Hubbard St',
 'Shore Drive & 55th St',
 'Wells St & Concord Pl',
 'Streeter Dr & Illinois St',
 'Ravenswood Ave & Montrose Ave',
 'Michigan Ave & 16th St',
 'Paulina  Ave (Wood St) & North Ave',
 'Green St (Halsted St) & Madison St',
 'Halsted St & Blackhawk St',
 'Aberdeen St & Madison St',
 'Princeton Ave & 31st St',
 'Aberdeen St & Monroe St (Madison St)',
 'Halsted St & James M Rochford St',
 'California Ave & Roosevelt Rd',
 'Halsted St & Madison St',
 'Pulaksi Rd & Eddy St',
 'Wood St & North Ave']

In [19]:
#adding co-ordinates for the removed stations in 2013
rm15 = {k: locations[k] for k in locations.keys() & set(removed15)}

#printing out the first item in the dictionary to check the key-value pair
list(rm15.items())[0]

('Sangamon St & Washington Blvd', {'lat': 41.88316500000001, 'long': -87.6511})

In [20]:
#Converting the dictionary into dataframe
rm15 = pd.DataFrame.from_dict(rm15, orient='index')

In [21]:
#Resetting the index
rm15.reset_index(inplace=True)

#Renaming the index column to Station name
rm15 = rm15.rename(columns = {'index':'Station Name'})

In [22]:
#Adding a column with values of color for every station
rm15['color'] = rm15.apply(lambda row: 'green', axis=1)

#printing to check the structure of the dataframe
rm15.head(1)

,Station Name,lat,long,color
0,Sangamon St & Washington Blvd,41.883165,-87.6511,green


# Location Marker map for interactivity purpose (Station Name)

In [23]:
#merging both the added16 and removed15 dataframes
merged = pd.concat([rm15, st16], axis =0)

In [24]:
#setting marker as the color mentioned in the color column of the dataframe
marker = merged.loc[merged['color']!='']

map = folium.Map(location=[marker.lat.mean(), marker.long.mean()], zoom_start=12,
                     width=980,height=590, control_scale=True, tiles='OpenStreetMap')
marker.apply(lambda row:folium.Marker(location=[row["lat"], row["long"]],
        popup=row['Station Name'],
          icon=folium.Icon(color=row['color'])
            ).add_to(map), axis=1)

map

# Circle marker map 2016
## Saving the map for converting into gif

In [25]:
#create a map
m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='cartodb positron')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2016')   
m.get_root().html.add_child(folium.Element(title_html))

#creates a CircleMarker and adds it to the map
def plotDot(point):
    folium.CircleMarker(location=[point.lat, point.long],             
                        color= "red",
                        radius=3,
                        weight=2
                       ).add_to(m)

#use df.apply(,axis=1) to "iterate" through every row in your dataframe
st16.apply(plotDot, axis = 1)

m

In [26]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [27]:
m.save('Gif_circlemarker/2016.html')

In [28]:
with fileinput.FileInput('Gif_circlemarker/2016.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [29]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_circlemarker')

fn='2016.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2016.png')
browser.quit()

In [30]:
image = Image.open('ss2016.png')
box = (0, 0, 1100, 700)
cropped_image = image.crop(box)
cropped_image.save('croppedss2016.png')

# Heatmap 2016
## Saving the map for converting into gif

In [31]:
numlist = st16.reset_index()[['lat', 'long']].values.tolist()


m = folium.Map(location=[41.761605, -87.644161], zoom_start=10,
                     width=950,height=700, control_scale=True, tiles='OpenStreetMap')

title_html = '''
                 <h3 align="left" style="font-size:18px"><b>{}</b></h3>
                 '''.format(' &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp&nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp &nbsp    Year: 2016')   
m.get_root().html.add_child(folium.Element(title_html))


HeatMap(numlist).add_to(folium.FeatureGroup(name='Heat Map').add_to(m))
folium.LayerControl().add_to(m)

m

In [32]:
os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi')

In [33]:
m.save('Gif_heatmap/2016.html')

In [34]:
with fileinput.FileInput('Gif_heatmap/2016.html', inplace=True) as file:
        for line in file:
            print(line.replace('zoomControl: true', 'zoomControl: false'), end='')

In [35]:
# Convert html files to png (screenshot each html page)

# We use a delay because we dont want to take a screenshot of the browser before the map is loaded
delay=3

os.chdir('E:\DataVisualization\Assignments\divvy clone 2\cs627-divvy-viswanathduragasi\Gif_heatmap')

fn='2016.html'
tmpurl='file://{path}/{mapfile}'.format(path=os.getcwd(),mapfile=fn)

browser = webdriver.Chrome()
browser.get(tmpurl)

#Give the map tiles some time to load
time.sleep(delay)
browser.save_screenshot('ss2016.png')
browser.quit()

In [36]:
image = Image.open('ss2016.png')
box = (0, 0, 1250, 780)
cropped_image = image.crop(box)
cropped_image.save('croppedss2016.png')